In [10]:
import pandas as pd
import json
import numpy as np
import sys
from dtw import dtw

In [11]:
#algoritmo principale
def DTWDistance(signalA, signalB, channel):
    n,m = len(signalA), len(signalB)
    DTW = np.zeros(shape=(n, m))
    for i in range(0,n):
        DTW[i,0] = np.finfo(dtype=float).max
    for i in range(0,m):
        DTW[0,i] = np.finfo(dtype=float).max
    DTW[0,0] = 0
    
    for i in range(1,n):
        for j in range(1,m):
            const = abs(signalA[i][channel] - signalB[j][channel])
            DTW[i,j] = const + min(DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1])
    
    return DTW[n - 1, m - 1]

In [12]:
#Algoritmo di scaling
def new_scale_df(data_frame, gyro, accl, pres):
    
    def scale(value, max_, min_):
        return (value - min_) / (max_ - min_)
        
    copy = data_frame.copy() 
    
    for wIndex in range(len(copy)):
        for vIndex in range(len(copy[wIndex])):
            copy[wIndex][vIndex][0] = scale(copy[wIndex][vIndex][0], accl, -accl)
            copy[wIndex][vIndex][1] = scale(copy[wIndex][vIndex][1], accl, -accl)
            copy[wIndex][vIndex][2] = scale(copy[wIndex][vIndex][2], accl, -accl)
            copy[wIndex][vIndex][3] = scale(copy[wIndex][vIndex][3], pres, 0)
            copy[wIndex][vIndex][4] = scale(copy[wIndex][vIndex][4], gyro, -gyro)
            copy[wIndex][vIndex][5] = scale(copy[wIndex][vIndex][5], gyro, -gyro)
            copy[wIndex][vIndex][6] = scale(copy[wIndex][vIndex][6], gyro, -gyro)
    
    return copy

In [13]:
def new_shift_df(data_frame, gyro, accl, pres):
    
    def shift(value, shifter):
        return value + shifter
    
    copy = data_frame.copy()
    
    for wIndex in range(len(copy)):
        for vIndex in range(len(copy[wIndex])):
            copy[wIndex][vIndex][0] = shift(copy[wIndex][vIndex][0], accl)
            copy[wIndex][vIndex][1] = shift(copy[wIndex][vIndex][1], accl)
            copy[wIndex][vIndex][2] = shift(copy[wIndex][vIndex][2], accl)
            copy[wIndex][vIndex][4] = shift(copy[wIndex][vIndex][4], gyro)
            copy[wIndex][vIndex][5] = shift(copy[wIndex][vIndex][5], gyro)
            copy[wIndex][vIndex][6] = shift(copy[wIndex][vIndex][6], gyro)
    
    return copy

In [14]:
with open('../dati/scrittura_di_computer/scaled/Dario_scaled.json', 'r') as f:
    data_giovanni = json.load(f)
with open('../dati/scrittura_di_computer/scaled/Antonio_scaled.json', 'r') as f:
    data_dario = json.load(f)

In [19]:
#scaled_giovanni = new_scale_df(data_giovanni, 250, 2, 1024)
#scaled_dario = new_scale_df(data_dario, 250, 2, 1024)



ModuleNotFoundError: No module named 'mlpy'

In [16]:




value = [0,0,0,0]
for i in range(7):
    print('indice: ', i)
    l2_norm = lambda x, y: abs(x[i] - y[i])
    d1, _ , _ , _ = dtw(data_giovanni[1], data_dario[1], dist=l2_norm)
    print('giovanni vs dario',d1)
    d2, _ , _ , _ = dtw(data_giovanni[2], data_dario[2], dist=l2_norm)
    print('giovanni vs dario',d2)
    d3, _ , _ , _ = dtw(data_giovanni[1], data_giovanni[2], dist=l2_norm)
    print('giovanni vs giovanni',d3)
    d4, _ , _ , _ = dtw(data_dario[1], data_dario[2], dist=l2_norm)
    print('dario vs dario',d4)
    value[0] += d1
    value[1] += d2
    value[2] += d3
    value[3] += d4
value
    

#l2_norm = lambda x, y: (x[0] - y[0]) ** 2
#d1, cost_matrix, acc_cost_matrix, path = dtw(scaled_giovanni[1], scaled_dario[1], dist=l2_norm)
#d2, _ , _ , _ = dtw(scaled_giovanni[1], scaled_giovanni[2], dist=l2_norm)
#print(d1, d2)

indice:  0
giovanni vs dario 0.0048783296249999995
giovanni vs dario 0.0034944435000000026
giovanni vs giovanni 0.0026201786249999966
dario vs dario 0.0025949397499999993
indice:  1
giovanni vs dario 0.004203702
giovanni vs dario 0.003087586625000002
giovanni vs giovanni 0.003247955874999998
dario vs dario 0.00301758025
indice:  2
giovanni vs dario 0.004450192624999997
giovanni vs dario 0.002997985125
giovanni vs giovanni 0.003350098
dario vs dario 0.003321319000000002
indice:  3
giovanni vs dario 0.01031103515625
giovanni vs dario 0.0071611328125
giovanni vs giovanni 0.00657275390625
dario vs dario 0.003517578125
indice:  4
giovanni vs dario 0.0021460762959999995
giovanni vs dario 0.0018170381170000006
giovanni vs giovanni 2.540456200000002e-05
dario vs dario 3.0900728999999796e-05
indice:  5
giovanni vs dario 0.003351045822999999
giovanni vs dario 0.002710083995999999
giovanni vs giovanni 0.0033607786500000005
dario vs dario 0.0021697175829999997
indice:  6
giovanni vs dario 0.004467

[0.03380799226525, 0.024699468656500002, 0.022562558953249993, 0.017389394213]

In [17]:





# escludiamo la pressione per ora perché è costante nel tempo e tende a diminuire le distanze fra due utenti

In [18]:
matrix = np.zeros(shape=(len(scaled_dario),len(scaled_dario) + len(scaled_giovanni)))

for i in range(len(scaled_dario)):
    for j in range(len(scaled_dario)):
        for s in range(7):
            if s == 3: continue
            l2_norm = lambda x, y: abs(x[s] - y[s]) 
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_dario[j], dist=l2_norm)
            matrix[i, j] += d
    for j in range(len(scaled_giovanni)):
        for s in range(7):
            if s == 3: continue
            l2_norm = lambda x, y: abs(x[s] - y[s])
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_giovanni[j], dist=l2_norm)
            matrix[i, j + len(scaled_dario)] += d
    print(i)
            
matrix    

NameError: name 'scaled_dario' is not defined

In [ ]:
shifted_dario = new_scale_df(data_dario, 250, 2, 1024)
shifted_giovanni = new_scale_df(data_giovanni, 250, 2, 1024)

In [ ]:
matrix = np.zeros(shape=(len(shifted_dario),len(shifted_dario) + len(shifted_giovanni)))

for i in range(len(shifted_dario)):
    for j in range(len(shifted_dario)):
        l2_norm = lambda x, y: abs((sum(x) - x[3]) - (sum(y) - y[3])) 
        d, _ , _ , _ = dtw(shifted_dario[i], shifted_dario[j], dist=l2_norm)
        matrix[i, j] = d
    for j in range(len(shifted_giovanni)):
        l2_norm = lambda x, y: abs((sum(x) - x[3]) - (sum(y) - y[3]))
        d, _ , _ , _ = dtw(shifted_dario[i], shifted_giovanni[j], dist=l2_norm)
        matrix[i, j + len(shifted_dario)] = d
    print(i)
            
matrix

In [ ]:
matrix = np.zeros(shape=(len(scaled_dario),len(scaled_dario) + len(scaled_giovanni)))
matrix.fill(sys.maxsize)

for i in range(len(scaled_dario)):
    for j in range(len(scaled_dario)):
        for s in range(7):
            l2_norm = lambda x, y: abs(x[s] - y[s]) 
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_dario[j], dist=l2_norm)
            matrix[i, j] = min(matrix[i, j], d)
    for j in range(len(scaled_giovanni)):
        for s in range(7):
            l2_norm = lambda x, y: abs(x[s] - y[s])
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_giovanni[j], dist=l2_norm)
            matrix[i, j + len(scaled_dario)] = min(matrix[i, j + len(scaled_dario)], d)
    print(i)
            
matrix    

In [ ]:
d = pd.DataFrame(matrix)
for i in range(len(matrix)):
    print(i,':',(matrix[i]).argmax(), max(matrix[i]))

In [ ]:
d

In [ ]:
min_values, max_values = [], []
for i in range(len(matrix)):
    minv, maxv = (sys.maxsize, -1), (-sys.maxsize, -1)
    for j in range(len(matrix[i])):
        if i == j: 
            continue
        minv = (matrix[i][j], j) if minv[0] > matrix[i][j] else minv
        maxv = (matrix[i][j], j) if maxv[0] < matrix[i][j] else maxv
    min_values.append(minv)
    max_values.append(maxv)
    
for i in range(len(min_values)):
    print("index ", i, " min value: ", min_values[i])
        